# ***Tiempo en el mercado***

In [23]:
# Manejo de datos
import os # Directorios
import pandas as pd # Manipulación df
# Gráficas
import plotly.graph_objects as go #Para obtener librería usar: pip install plotly
from plotly.subplots import make_subplots
import plotly.io as pio # Exportar gráfica

In [7]:
# Obtener el directorio actual de trabajo
directorio_actual = os.getcwd()
# Directorio donde se encuentran los archivos JSON (ruta relativa)
directorio_json = os.path.join(directorio_actual, 'datos_json')
#print("Directorio JSON relativo:", directorio_json)
# Obtener la lista de archivos JSON en el directorio
archivos_json = os.listdir(directorio_json)

In [8]:
dataframes = {}
# Iterar sobre cada archivo JSON y crear un DataFrame
for archivo in archivos_json:
    # Obtener el nombre de la tabla del nombre del archivo
    nombre_tabla = archivo.replace('datos_', '').replace('.json', '')    
    # Cargar el archivo JSON en un DataFrame y asignarlo a una variable con un nombre dinámico
    ruta_json = os.path.join(directorio_json, archivo)
    globals()[f"df_{nombre_tabla}"] = pd.read_json(ruta_json)

In [9]:
# Obtener todos los nombres de las variables globales
nombres_variables_globales = list(globals().keys())
# Filtrar los nombres, solo aquellos que comienzan con "df_"
nombres_df = [nombre for nombre in nombres_variables_globales if nombre.startswith("df_")]

# Lista de nombres de los DataFrames creados
print("Lista de DataFrames creados:")
print(nombres_df)

Lista de DataFrames creados:
['df_alfa_q_feb_2023_pachuca', 'df_alfa_q_jul_2023_pachuca', 'df_alfa_q_jun_2023_pachuca', 'df_alfa_q_mar_2023_pachuca', 'df_alfa_q_nov_2022_pachuca', 'df_alfa_q_oct_2022_pachuca', 'df_alfa_q_sep_2023_pachuca', 'df_enero_2024_querertaro', 'df_financiamientos_2019_pachuca', 'df_financiamientos_2020_pachuca', 'df_financiamientos_2021_pachuca', 'df_financiamientos_2022_pachuca', 'df_financiamientos_2023_pachuca', 'df_grupos_edad_pachuca', 'df_grupos_edad_queretaro', 'df_junio_2023_queretaro', 'df_publicacion_pachuca', 'df_salarios_pachuca', 'df_usuarios']


______
# *Tiempo en el mercado*

In [11]:
df_mercado_pachuca = df_publicacion_pachuca[['id','tiempo_de_publicacion']]
df_mercado_pachuca

,id,tiempo_de_publicacion
0,1,72
1,2,119
2,3,293
3,4,293
4,5,273
...,...,...
2062,2094,320
2063,2095,320
2064,2096,27
2065,2097,26


In [12]:
print(df_mercado_pachuca.shape)
tiempo = df_mercado_pachuca['tiempo_de_publicacion'].unique()
print(tiempo)

(2067, 2)
['72' '119' '293' '273' '117' '116' '134' '365' '234' '272' '130' '323'
 '29' '64' '230' '152' '21' '97' '96' '131' '118' '121' '181' '51' '23'
 '101' '82' '237' '66' '6' '124' '49' '71' '99' '30' '9' '37' '257' ''
 '68' '76' '84' '176' '62' '59' '197' '111' '120' '1' '125' '85' '0' '306'
 '42' '41' '297' '3' '7' '182' '22' '200' '129' '259' '191' '73' '140'
 '100' '24' '89' '86' '83' '348' '10' '250' '98' '240' '79' '187' '48'
 '215' '104' '90' '180' '123' '146' '35' '107' '216' '61' '40' '88' '94'
 '93' '343' '359' '161' '177' '36' '39' '17' '16' '15' '105' '13' '63'
 '156' '356' '154' '346' '87' '2' '92' '127' '322' '28' '26' '60' '70' '8'
 '247' '246' '252' '296' '78' '327' '351' '336' '34' '14' '139' '133'
 '300' '256' '122' '225' '224' '19' '208' '141' '54' '232' '56' '47' '302'
 '238' '275' '280' '292' '324' '294' '77' '81' '112' '69' '91' '333' '334'
 '218' '27' '291' '135' '229' '266' '165' '242' '289' '202' '251' '328'
 '32' '128' '313' '138' '307' '260' '53' '337' 

Es importante destacar que los valores mayores a 365 no hay en la base

### Asignar rangos

In [13]:
# Definir la función de categorización
def categorizar(valor):
    if valor.isdigit():  # Verificar si el valor es un número
        valor = int(valor)  # Convertir a entero si es un número
        if 0 <= valor <= 10:
            return '0-10'
        elif 11 <= valor <= 30:
            return '11-30'
        elif 31 <= valor <= 100:
            return '31-100'
        elif 101 <= valor <= 364:
            return '101-364'
        else:
            return '>=365'
    else:
        return 'No numérico'  # Manejar valores no numéricos
# Lista de nombres de los DataFrames
dataframes_name = ['df_mercado_pachuca']

# Lista de DataFrames
dataframes_list = [df_mercado_pachuca]

# Definir el orden de las categorías
todas_categorias = ['0-10', '11-30', '31-100', '101-364', '>=365']

# Diccionario para almacenar los DataFrames de conteo por nombre de DataFrame original
dataframes_conteo_por_nombre = {}

# Iterar sobre cada DataFrame
for name, df in zip(dataframes_name, dataframes_list):
    # Aplicar la función de categorización
    df['dias'] = df['tiempo_de_publicacion'].apply(categorizar)
    # Contar el número de registros únicos por cada valor único en la columna 'dias'
    conteo_por_dias = df['dias'].value_counts().reindex(todas_categorias, fill_value=0)
    # Crear un nuevo DataFrame de conteo
    df_conteo = pd.DataFrame({'dias': conteo_por_dias.index, 'conteo': conteo_por_dias.values})
    # Ordenar el DataFrame según el orden definido en todas_categorias
    df_conteo['dias'] = pd.Categorical(df_conteo['dias'], categories=todas_categorias, ordered=True)
    df_conteo = df_conteo.sort_values(by='dias')
    # Almacenar el DataFrame de conteo en el diccionario por nombre de DataFrame original
    dataframes_conteo_por_nombre[name] = df_conteo
    # Imprimir el nombre del DataFrame de conteo y su contenido
    nuevo_nombre_df_conteo = 'conteo_' + name
    globals()[nuevo_nombre_df_conteo] = df_conteo
    print(f'Nombre del DataFrame de conteo: {nuevo_nombre_df_conteo}')
    print(df_conteo)


Nombre del DataFrame de conteo: conteo_df_mercado_pachuca
      dias  conteo
0     0-10     162
1    11-30     169
2   31-100     418
3  101-364     905
4    >=365     411


C:\Users\yoe11\AppData\Local\Temp\ipykernel_9180\3770229744.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['dias'] = df['tiempo_de_publicacion'].apply(categorizar)


In [14]:
conteo_df_mercado_pachuca

,dias,conteo
0,0-10,162
1,11-30,169
2,31-100,418
3,101-364,905
4,>=365,411


In [15]:
values = conteo_df_mercado_pachuca['conteo']
labels = conteo_df_mercado_pachuca['dias']
colores = ['#3d45c0', '#ff4853', '#ffb038', '#7cc668', '#67c2df', '#6773df', '#939fad']

fig = go.Figure()
fig.add_trace(go.Bar(
    x=labels, 
    y=values, 
    marker_color=colores,  # Especifica los colores de las barras
    text=values,  # Texto que se mostrará en las barras 
    textposition='inside',  # Posición del texto (puede ser 'inside' o 'outside')
))
fig.update_layout(
    title='Número de unidades por días en el mercado',  # Título de la gráfica
    xaxis=dict(title='Rango días'),  # Título del eje x
    yaxis=dict(
        #title='Cantidad', # Título del eje y
        gridcolor='#dddcda', # Color de las líneas que dividen los rangos del eje Y
         gridwidth=1,
    ),  
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
)
# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

# Exportar
guardar_grafico_como_html(fig, 'g_bar_tiempomercado_pachuca', carpeta='graficas')
fig.show()

___
# ***3.8 Riesgo.***

In [16]:
conteo_df_mercado_pachuca

,dias,conteo
0,0-10,162
1,11-30,169
2,31-100,418
3,101-364,905
4,>=365,411


In [18]:
# Definir los valores para la columna 'stock'
stock_values = []
for dias in conteo_df_mercado_pachuca['dias']:
    if dias == '>=365':
        stock_values.append('Más de un año en stock')
    else:
        stock_values.append('Menos de un año en stock')

# Crear el DataFrame 'riesgo'
riesgo = pd.DataFrame({
    'stock': stock_values,
    'casas': conteo_df_mercado_pachuca['conteo']
})
riesgo

,stock,casas
0,Menos de un año en stock,162
1,Menos de un año en stock,169
2,Menos de un año en stock,418
3,Menos de un año en stock,905
4,Más de un año en stock,411


In [21]:
# Agrupar por la columna 'stock' y sumar los valores de 'casas'
riesgo_stock = riesgo.groupby('stock').sum().reset_index()
# Ordenar el DataFrame para que 'Menos de un año en stock' quede abajo
riesgo_stock = riesgo_stock.sort_values(by='stock', ascending=False)
riesgo_stock

,stock,casas
1,Más de un año en stock,411
0,Menos de un año en stock,1654


In [5]:
datos = {
    'stock': ['Más de un año en stcok','Menos de un año en stock'],
    'casas': [ 413, 2067 ]
}
# Crear DataFrame
df_riesgo_pachuca = pd.DataFrame(datos)
df_riesgo_pachuca

,stock,casas
0,Más de un año en stcok,413
1,Menos de un año en stock,2067


In [22]:
labels = riesgo_stock['stock']
values = riesgo_stock['casas']
colores = ['#3d45c0', '#ff4853']
# Crear el gráfico de pastel
fig = go.Figure(data=[go.Pie(
    labels=labels, 
    values=values, 
    sort=False,    
    marker=dict(colors=colores,)
)])
fig.update_layout(title="% de riesgo en stock")
# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

# Exportar
guardar_grafico_como_html(fig, 'g_pie_riesgo_pachuca', carpeta='graficas')
fig.show()

___
# ***10. Proyección de stock en dias***

In [9]:
# Datos a agregar
datos_nuevos = {
    'Mes': ['julio 2023', 'agosto 2023', 'septiembre 2023', 'octubre 2023', 'noviembre 2023', 'diciembre 2023'],
    'Dias': [165, 155, 145, 150, 148, 149]
}

# Convertir a DataFrame
stock_proyeccion_dias = pd.DataFrame(datos_nuevos)
stock_proyeccion_dias

,Mes,Dias
0,julio 2023,165
1,agosto 2023,155
2,septiembre 2023,145
3,octubre 2023,150
4,noviembre 2023,148
5,diciembre 2023,149


In [17]:
import plotly.graph_objects as go

# Obtener los datos
dias = stock_proyeccion_dias['Dias']
meses = stock_proyeccion_dias['Mes']

# Crear figura
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=meses,
    y=dias,
    mode='markers+lines',
    marker=dict(color='blue'),
))
# Agregar anotaciones para mostrar los valores encima de los puntos
for mes, dia in zip(meses, dias):
    fig.add_annotation(
        x=mes,
        y=dia,
        text=str(dia),  # Texto que se mostrará (el valor del día)
        showarrow=False,  # No mostrar la flecha
        font=dict(color='black', size=12),  # Estilo del texto
        xshift=0,  # Desplazamiento horizontal del texto
        yshift=17,  # Desplazamiento vertical del texto
        textangle=0  # Ángulo del texto
    )
# Actualizar diseño
fig.update_layout(
    title='Proyección de días en stock',
    xaxis_title='Mes',
    yaxis_title='Días',
    xaxis=dict(gridcolor='#dddcda', tickmode='array', tickvals=list(range(len(meses))), ticktext=meses),
    yaxis=dict(gridcolor='#dddcda', gridwidth=1),
    plot_bgcolor='rgba(0,0,0,0)'
)

# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

guardar_grafico_como_html(fig, 'g_scatt_stockdias_pachuca', carpeta='graficas')
fig.show()

___
# ***11. Meses de oferta (Month of Supply)***

In [19]:
import pandas as pd

# Datos
datos = {
    'meses_oferta': ['noviembre 2022', 'febrero 2023', 'marzo 2023', 'junio 2023', 'septiembre 2023', 'diciembre 2023'],
    'promedio' : [16.69,23.05,31.35,37.89,28.26,16.72]
}

# Convertir a DataFrame
df_oferta_creditos = pd.DataFrame(datos)
df_oferta_creditos

,meses_oferta,promedio
0,noviembre 2022,16.69
1,febrero 2023,23.05
2,marzo 2023,31.35
3,junio 2023,37.89
4,septiembre 2023,28.26
5,diciembre 2023,16.72


In [20]:
import plotly.graph_objects as go

# Obtener los datos
dias = df_oferta_creditos['promedio']
meses = df_oferta_creditos['meses_oferta']

# Crear figura
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=meses,
    y=dias,
    mode='markers+lines',
    marker=dict(color='blue'),
))
# Agregar anotaciones para mostrar los valores encima de los puntos
for mes, dia in zip(meses, dias):
    fig.add_annotation(
        x=mes,
        y=dia,
        text=str(dia),  # Texto que se mostrará (el valor del día)
        showarrow=False,  # No mostrar la flecha
        font=dict(color='black', size=12),  # Estilo del texto
        xshift=0,  # Desplazamiento horizontal del texto
        yshift=17,  # Desplazamiento vertical del texto
        textangle=0  # Ángulo del texto
    )
# Actualizar diseño
fig.update_layout(
    title='Meses de oferta',
    xaxis_title='Mes de oferta créditos',
    yaxis_title='Promedio',
    xaxis=dict(gridcolor='#dddcda', tickmode='array', tickvals=list(range(len(meses))), ticktext=meses),
    yaxis=dict(gridcolor='#dddcda', gridwidth=1),
    plot_bgcolor='rgba(0,0,0,0)'
)

# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

guardar_grafico_como_html(fig, 'g_scatt_mesesoferta_pachuca', carpeta='graficas')
fig.show()

____
# ***3.11 Cambio de Precios en el Mercado Inmobiliario de Pachuca por Mes.***

In [38]:
datos_cambio = {
    'porcentage': [-13, 15, -12, 12, -2, 2, -15],
    'year_month': ['2022,11', '2023,2', '2023,3', '2023,6', '2023,7', '2023,9', '2023,12',]
}
# Crear DataFrame
df_cambio = pd.DataFrame(datos_cambio)
df_cambio

,porcentage,year_month
0,-13,"2022,11"
1,15,"2023,2"
2,-12,"2023,3"
3,12,"2023,6"
4,-2,"2023,7"
5,2,"2023,9"
6,-15,"2023,12"


In [40]:
import plotly.graph_objects as go

colores = ['#3d45c0', '#ff4853', '#ffb038', '#7cc668', '#67c2df', '#6773df', '#939fad']

fig = go.Figure()
fig.add_trace(go.Bar(
    x=df_cambio['year_month'],
    y=df_cambio['porcentage'],
    base=0,
    marker_color=[colores[i % len(colores)] for i in range(len(df_cambio))],  # Usar los colores definidos
    text=df_cambio['porcentage'],  # Usar los porcentajes como texto
    textposition='inside',  # Mostrar el texto dentro de las barras
    texttemplate='%{text}%',  # Formato del texto
    hoverinfo='none',  # No mostrar información al pasar el cursor
))
fig.update_layout(
    title='Cambio de Precios en el Mercado inmobiliario de Pachuca por mes',
    xaxis_title='Año-Mes',
    yaxis_title='%',
    yaxis=dict(gridcolor='#dddcda'),
    plot_bgcolor='rgba(0,0,0,0)'
)
# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

guardar_grafico_como_html(fig, 'g_bar_precioscambiomes_pachuca', carpeta='graficas')

fig.show()